In [1]:
from analysis_utils import *
%load_ext autoreload
%reload_ext autoreload

import numpy as np
import pandas as pd

ptRNA = pd.read_excel('tRNAValues.xlsx',header=None)
gr04_ptRNA=ptRNA[0]
gr07_ptRNA=ptRNA[1]
gr107_ptRNA=ptRNA[2]
gr16_ptRNA=ptRNA[3]
gr25_ptRNA=ptRNA[4]

pCodon = pd.read_excel('codonValues.xlsx',header=None)
gr04_pCodon = pCodon[0]
gr07_pCodon = pCodon[1]
gr107_pCodon = pCodon[2]
gr16_pCodon = pCodon[3]
gr25_pCodon = pCodon[4]

rxndiff=dict()

In [6]:
def computeBiasedElongationLatency(biasVal=1):
    sim_time_all = list()
    sim_time_std_all = list()
    transp_time_all = list()
    rxn_time_all = list()
    pcogrib_all = list()
    
    rib_gr = [4,8,9,9,8,7]
    ptRNA = [gr07_ptRNA,gr107_ptRNA,gr16_ptRNA,gr16_ptRNA,gr25_ptRNA,gr25_ptRNA]
    pCodon =  [gr07_pCodon,gr107_pCodon,gr16_pCodon,gr16_pCodon,gr25_pCodon,gr25_pCodon]

    for i,num_rib in enumerate(rib_gr):
        weight = cognateDistrib(ptRNA[i],pCodon[i])
        sim_time_gr = list()
        sim_time_gr_var=list()
        rxn_count_gr = list()
        transp_latency_gr = list()
        rxn_latency_gr = list()
        pcogrib_gr = list()

        num_tRNA=42
        for num_tRNA in np.arange(1,42):
            sim_times=list()
            rxn_count = list()
            transp_latency = list()
            rxn_latency = list()
            N=1000
            for i in range(N):
                rxn = eventbased_sim(rib_num=num_rib,tRNA_cog=num_tRNA,repeatAllowed=True,bias=biasVal)
                sim_times.append(rxn[0])
                rxn_count.append(rxn[1])
                rxn_latency.append(rxn[2])
                transp_latency.append(rxn[0]-rxn[2])

            sim_time_gr.append(np.average(sim_times))
            rxn_count_gr.append(np.average(rxn_count))
            sim_time_gr_var.append(np.std(sim_times)**2)
            pcogrib_gr.append(rxn[3])
            
            transp_latency_gr.append(np.average(transp_latency))
            rxn_latency_gr.append(np.average(rxn_latency))

        #print(np.sum(sim_time_gr*weight[1:len(sim_time_gr)+1]))
        sim_time_all.append(np.sum(sim_time_gr*weight[1:len(sim_time_gr)+1]))
        sim_time_std_all.append(np.sqrt(np.sum((sim_time_gr_var)*weight[1:len(sim_time_gr)+1]))/np.sqrt(num_tRNA*N))
        transp_time_all.append(np.sum(transp_latency_gr*weight[1:len(transp_latency_gr)+1]))
        rxn_time_all.append(np.sum(rxn_latency_gr*weight[1:len(rxn_latency_gr)+1]))
        pcogrib_all.append(np.sum(pcogrib_gr*weight[1:len(rxn_latency_gr)+1]))
        
    return sim_time_all, sim_time_std_all, transp_time_all, rxn_time_all,pcogrib_all

In [ ]:
[36.21142494192998,  17.357535886747577, 11.730928900813964, 10.107698330956518,  7.799635735314417,  5.761090774097384]
[0.495518389669115,0.3065313307595597,0.16363724964738255,0.1467234788512787,0.11842922758156244,0.09412528299436541]
[32.859535130295335,14.997047110896354,10.152033718185963,8.634526845284789,6.581741747056677,4.6762812782310945]
[3.3518898116346367,2.3604887758512163,1.5788951826279987,1.4390199270843849,1.2178939882577413,1.0848094958662904]
[1,2,3,3.5,4.5,7]

[1, 2, 3, 3.5, 4.5, 7]

In [7]:
elongLatencyList = list()
elongLatency_STDlist = list()
transportLatencyList=list()
reactionLatencyList=list()
bias = np.arange(1,8,0.5)

for biasVal in bias:
    elongLatency, elongLatency_STD, tRNA_transport,tRNA_reaction,pcogrib = computeBiasedElongationLatency(biasVal)
    elongLatencyList.append(elongLatency)
    elongLatency_STDlist.append(elongLatency_STD)
    transportLatencyList.append(tRNA_transport)
    reactionLatencyList.append(tRNA_reaction)
    print("\n\nBias: ", biasVal, "\n", elongLatency, elongLatency_STD, "\n", tRNA_transport, "\n",tRNA_reaction)



Bias:  1.0 
 [35.930587390775145, 37.55699003318843, 35.97726789020362, 36.121566846595044, 37.611632485851565, 35.838066896273375] [0.23509077560780514, 0.24750870161194657, 0.24841399556582114, 0.23757320231170767, 0.24620696350238142, 0.23074272020614625] 
 [32.5247109147635, 30.3519121077561, 28.26084113906479, 28.47079491006946, 30.469048541032894, 29.833700067207644] 
 [3.405876476011637, 7.205077925432331, 7.716426751138842, 7.650771936525576, 7.142583944818675, 6.004366829065734]


Bias:  1.5 
 [24.27026298946842, 23.600863905358256, 23.400547745780553, 23.388882616724167, 23.165692261712493, 23.964044804739334] [0.15947293569679902, 0.1522183778350832, 0.1495562684830542, 0.15633616824777175, 0.15303154667728727, 0.16389127862713382] 
 [22.256284398663013, 20.024953872542063, 19.33972406861411, 19.373208996193256, 19.606588033879003, 20.683855656876492] 
 [2.0139785908054106, 3.5759100328161937, 4.060823677166432, 4.015673620530911, 3.55910422783349, 3.2801891478628424]


Bi

In [2]:
import numpy as np
print([1000/12,1000/15,1000/18,1000/19,1000/20,1000/21]-np.ones(6)*(1000/1529+1000/209+1000/200+1000/32))
#1
#1.5
#2.5
#3.5
#4.5
#6.5

p_cogRib=[bias*1/42]*tRNA_cog
p_cogRib=np.concatenate((p_cogRib,[1/42]*(42-tRNA_cog)))
p_cogRib = p_cogRib/sum(p_cogRib)

[41.6446221  24.97795543 13.86684432 10.94286772  8.31128877  5.93033639]


In [7]:
elongLatencyList = list()
elongLatency_STDlist = list()
transportLatencyList=list()
reactionLatencyList=list()
bias = np.arange(1,8,0.5)

for biasVal in bias:
    elongLatency, elongLatency_STD, tRNA_transport,tRNA_reaction,pcogrib = computeBiasedElongationLatency(biasVal)
    elongLatencyList.append(elongLatency)
    elongLatency_STDlist.append(elongLatency_STD)
    transportLatencyList.append(tRNA_transport)
    reactionLatencyList.append(tRNA_reaction)
    print("\n\nBias: ", biasVal, "\n", elongLatency, elongLatency_STD, "\n", tRNA_transport, "\n",tRNA_reaction,"\n",pcogrib)



Bias:  1.0 
 [37.55369102066618, 37.49948145305508, 36.96692009009822, 36.17816640526125, 34.95225145399006, 36.926179991576355] [0.24889287616063552, 0.25112967891183013, 0.24506298102378982, 0.2318693917252581, 0.23944370814903732, 0.25011617182843127] 
 [34.01663109406401, 30.41369509254369, 29.154101488155067, 28.491437978874004, 28.409508968849106, 30.805382022214264] 
 [3.537059926602166, 7.085786360511388, 7.812818601943163, 7.686728426387252, 6.542742485140954, 6.120797969362092] 
 [0.044093442947946836, 0.04334646794784338, 0.04616656233030936, 0.04515334051159743, 0.04738491780503308, 0.048690305303684046]


Bias:  1.5 
 [24.780336489925332, 23.572175649670893, 24.04143416210719, 23.720800439751226, 24.261451540730253, 22.724787034886003] [0.169974108591396, 0.15394514864352532, 0.15592611749581273, 0.15130735362011744, 0.162983305176057, 0.1548896742407016] 
 [22.773986335365365, 20.007025364980883, 19.924346097144543, 19.62002116177687, 20.57282321044486, 19.6446987850517

In [ ]:
#1
#1.5
#2.5
#3.5
#






#1 34.01663109406401  3.537059926602166
#1.5 20.007025364980883  3.5651502846900125
#2.5  11.68166201748362  2.0426277448724632
#3.5 8.274169505141892 1.4374991110185171
#4.5 7.074660404155837 1.2407775100632652
#6.5 4.926203750197745 1.025038678927513

[34.01663109406401,20.007025364980883,11.68166201748362,8.274169505141892, 7.074660404155837,  4.926203750197745]
[3.537059926602166,3.5651502846900125,2.0426277448724632,1.4374991110185171,1.2407775100632652,1.025038678927513]

In [9]:
print(34.01663109406401/(42+3.537059926602166))
print(20.007025364980883 /(42+3.5651502846900125))
print(11.68166201748362 /(42+2.0426277448724632))
print(8.274169505141892/(42+1.4374991110185171))


0.747009823403024
0.43908612700665856
0.2652353552824428
0.19048448171462604


In [12]:
np.array([1000/12,1000/15,1000/18,1000/19,1000/20,1000/21])-np.array([54.275398761814074, 69.9277178408021, 64.23009633006572, 58.552181767860304, 54.873088832469165, 51.2858285896607])

array([29.05793457, -3.26105117, -8.67454077, -5.92060282, -4.87308883,
       -3.66678097])

In [13]:
1000/21

47.61904761904762